In [1]:
pip install torch transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 4.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install nlpaug nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 4.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/moesanjari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/moesanjari/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your CSV
df = pd.read_csv("data/balanced_dataset.csv")  # Replace with your file path

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split
train_test = dataset.train_test_split(test_size=0.2, seed=42)  # Seed for reproducibility
train_val = train_test["train"].train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    "train": train_val["train"],
    "validation": train_val["test"],
    "test": train_test["test"]
})

print(dataset)  # Check sizes
print(dataset["train"][0])  # View a sample

DatasetDict({
    train: Dataset({
        features: ['SMSes', 'Labels'],
        num_rows: 37695
    })
    validation: Dataset({
        features: ['SMSes', 'Labels'],
        num_rows: 4189
    })
    test: Dataset({
        features: ['SMSes', 'Labels'],
        num_rows: 10472
    })
})
{'SMSes': 'At the moment only jazz and ufone are offering esim.', 'Labels': 0.0}


In [39]:
# Rename the label column to the expected name
if "Labels" in dataset["train"].column_names:
    dataset = dataset.rename_column("Labels", "labels")

# Verify
print("Updated columns:", dataset["train"].column_names)  # Should now include 'labels' instead of 'Labels'
print("Sample:", dataset["train"][0])  # Check that labels are present (e.g., 0 or 1)

Updated columns: ['SMSes', 'labels']
Sample: {'SMSes': 'At the moment only jazz and ufone are offering esim.', 'labels': 0.0}


In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import evaluate

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [19]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.config.problem_type = "single_label_classification"

In [43]:
def tokenize_function(examples):
    return tokenizer(examples["SMSes"], padding="max_length", truncation=True, max_length=256)  # Assuming text column is "SMSes" based on prior error

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format (as previously advised)
columns_to_keep = ["input_ids", "attention_mask", "labels"]
tokenized_datasets = tokenized_datasets.remove_columns(
    [col for col in tokenized_datasets["train"].column_names if col not in columns_to_keep]
)
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/37695 [00:00<?, ? examples/s]

Map:   0%|          | 0/4189 [00:00<?, ? examples/s]

Map:   0%|          | 0/10472 [00:00<?, ? examples/s]

In [55]:
def convert_labels(examples):
    # Convert to int/long to avoid float inference
    examples["labels"] = [int(label) for label in examples["labels"]]
    return examples

tokenized_datasets = tokenized_datasets.map(convert_labels, batched=True)

Map:   0%|          | 0/37695 [00:00<?, ? examples/s]

Map:   0%|          | 0/4189 [00:00<?, ? examples/s]

Map:   0%|          | 0/10472 [00:00<?, ? examples/s]

In [45]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Test on 5 samples
sample_texts = [example["SMSes"] for example in dataset["test"].select(range(5))]
base_predictions = classifier(sample_texts)
print("Base model predictions:", base_predictions)

Device set to use mps:0


Base model predictions: [{'label': 'LABEL_1', 'score': 0.5129286050796509}, {'label': 'LABEL_0', 'score': 0.5274924635887146}, {'label': 'LABEL_0', 'score': 0.5380409955978394}, {'label': 'LABEL_0', 'score': 0.5582281351089478}, {'label': 'LABEL_0', 'score': 0.5798953175544739}]


In [59]:
import evaluate
from transformers import Trainer, TrainingArguments  # Add this import

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels)["f1"]
    }

# Evaluation setup
eval_args = TrainingArguments(
    output_dir="./eval",
    per_device_eval_batch_size=16,
    no_cuda=not torch.cuda.is_available()  # Use CPU if no GPU
)
evaluator = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)
base_results = evaluator.evaluate()
print("Base model results:", base_results)

/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Base model results: {'eval_model_preparation_time': 0.0007, 'eval_runtime': 220.2048, 'eval_samples_per_second': 47.556, 'eval_steps_per_second': 2.975}


In [63]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/37695 [00:00<?, ? examples/s]

Map:   0%|          | 0/4189 [00:00<?, ? examples/s]

Map:   0%|          | 0/10472 [00:00<?, ? examples/s]

In [67]:
from transformers import TrainingArguments, Trainer

# Set problem_type explicitly (as before)
model.config.problem_type = "single_label_classification"

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Fine for CUDA; MPS uses bfloat16 automatically
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    dataloader_pin_memory=False,  # Fix for MPS warning (replaces pin_memory)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,  # Fix deprecation warning (replaces tokenizer=)
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 